In [29]:
import torch
from torch import nn
import torchvision
from torchvision import transforms

In [30]:
# insert in utils 
def accuracy_fn(pred, y):
    return (pred.argmax(dim=1) == y).float().mean()

In [31]:
# insert in utils 

def report(loss, pred, y):
    print(f'{loss:.2f}, {accuracy_fn(preds, yb):.2f}')

In [32]:
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

In [33]:
from utils import dataloader
BATCH_SIZE = 64
train_dataloader, test_dataloader, class_names = dataloader(train_dataset, test_dataset, BATCH_SIZE)

In [34]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [35]:
from conv import *
cnn = nn.Sequential(
    conv(1, 4),
    conv(4, 8),
    conv(8, 16),
    conv(16, 16),
    conv(16, 10, act=False),
    nn.Flatten()
)

In [37]:
lr = 0.4

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=cnn.parameters(), lr=lr)

In [50]:
# insert in train 

def fit(epochs, model, train_dataloader, test_dataloader, loss_fn, optimizer, device):
    for epoch in range(epochs):
        model.to(device)
        model.train()
        for X, y in train_dataloader:
            X, y = X.to(device), y.to(device)
            loss = loss_fn(model(X), y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.inference_mode():
            tot_loss, tot_acc, count = 0.,0.,0
            for X, y in test_dataloader:
                X, y = X.to(device), y.to(device)
                pred = model(X)
                n = len(X)
                count += n
                tot_loss += loss_fn(pred, y).item()*n
                tot_acc += accuracy_fn(pred, y).item()*n
        print("--------------------")
        print(count)
        print(tot_loss)
        print(tot_acc)
        print("--------------------")
        print(epoch, tot_loss/count, tot_acc/count)
        print("--------------------")
    return tot_loss/count, tot_acc/count

In [51]:
fit(epochs=3, model=cnn, loss_fn=loss_fn, optimizer=optimizer, train_dataloader=train_dataloader, test_dataloader=test_dataloader, device=device)

--------------------
10000
4014.6596205234528
8445.0
--------------------
0 0.40146596205234525 0.8445
--------------------
--------------------
10000
4451.583886146545
8364.0
--------------------
1 0.44515838861465457 0.8364
--------------------
--------------------
10000
4410.060415506363
8392.0
--------------------
2 0.4410060415506363 0.8392
--------------------


(0.4410060415506363, 0.8392)

In [52]:
from tqdm import tqdm
def fit_2(epochs, model, train_dataloader, test_dataloader, loss_fn, optimizer, device):
    model.to(device)
    
    train_count, test_count = 0, 0
    train_loss, train_acc, test_loss, test_acc = 0, 0, 0, 0
    
    for epoch in tqdm(range(epochs)):
        model.train()
        for X, y in train_dataloader:
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred, y) 
            train_loss += loss
            train_acc += accuracy_fn(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.inference_mode():
            for X, y in test_dataloader:
                X, y = X.to(device), y.to(device)
                test_pred = model(X)
                test_loss += loss_fn(test_pred, y)
                test_acc += accuracy_fn(pred=test_pred, y=y)
        
        train_count = len(train_dataloader)*(epoch+1)
        test_count = len(test_dataloader)*(epoch+1)
        
        print(f"| Epoch: {epoch} | Train loss: {train_loss / train_count } | Train accuracy: {train_acc / train_count} | Test loss: {test_loss / test_count} | Test accuracy: {test_acc / test_count} |")
    return (
        train_loss.item() / train_count,
        train_acc / train_count,
        test_loss.item() / test_count,
        test_acc / test_count
    )

In [53]:
fit_2(epochs=3, model=cnn, loss_fn=loss_fn, optimizer=optimizer, train_dataloader=train_dataloader, test_dataloader=test_dataloader, device=device)

 33%|███▎      | 1/3 [00:06<00:13,  6.83s/it]

| Epoch: 0 | Train loss: 0.34500396251678467 | Train accuracy: 0.8720682263374329 | Test loss: 0.49899420142173767 | Test accuracy: 0.8213574886322021 |


 67%|██████▋   | 2/3 [00:13<00:06,  6.81s/it]

| Epoch: 1 | Train loss: 0.34374773502349854 | Train accuracy: 0.8717517256736755 | Test loss: 0.4580437242984772 | Test accuracy: 0.8358380198478699 |


100%|██████████| 3/3 [00:20<00:00,  6.74s/it]

| Epoch: 2 | Train loss: 0.3419395089149475 | Train accuracy: 0.8724180459976196 | Test loss: 0.4411463737487793 | Test accuracy: 0.8426221013069153 |


(0.34193950726279315,
 tensor(0.8724, device='cuda:0'),
 0.4411463646372412,
 tensor(0.8426, device='cuda:0'))